In [ ]:
## This file implements neural networks and logistic regression on p002ypresabsSTCC_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 85.51% for combination data, and 86.39% for over-sampling data.
## To improve accuracy, we use lasso to select important features and thus have new training and test datasets.
## For the new neural networks on these new datasets, the accuracy is 87.30% for combination data and 82.31% for over-sampling data.
## For logistic regression, the accuracies are 100% for both combination and over-sampling data.
## Since the accuracy scores are pretty high in logistic regression, we further construct random forest models, which 
## are relatively less likely to bring overfitting compared to decision tree.
## For random forest, the accuracies are again 100% for both combination and over-sampling data.
## For random forest with cross-validation, the mean accuracy is 99.68% for combination data, and 98.83% for over-sampling.

In [1]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p002ypresabsSTCC_quant.csv')
df.shape

(255, 1761)

In [2]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [3]:
df['pheno']

0      0.194875
1      0.265250
2      0.440625
3      0.175500
4      0.173625
5      0.270375
6      0.171000
7      0.182500
8      0.278875
9      0.174375
10     0.176250
11     0.411125
12     0.178250
13     0.191500
14     0.164875
15     0.181875
16     0.394125
17     0.364875
18     0.409875
19     0.191875
20     0.236125
21     0.277000
22     0.169875
23     0.171625
24     0.523625
25     0.169500
26     0.381000
27     0.170000
28     0.160750
29     0.389750
         ...   
225    0.230500
226    0.279800
227    0.319500
228    0.424100
229    0.460500
230    0.304750
231    0.193875
232    0.243000
233    0.195750
234    0.184750
235    0.329125
236    0.160000
237    0.157500
238    0.176375
239    0.326750
240    0.355750
241    0.281375
242    0.181875
243    0.174125
244    0.359250
245    0.161375
246    0.185875
247    0.172750
248    0.167750
249    0.179875
250    0.365375
251    0.223500
252    0.170750
253    0.261125
254    0.231000
Name: pheno, Length: 255

In [59]:
df.head()

,id,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATTTTCAAATCCCAT,...,group_8665,group_8666,group_8667,group_8836,group_8913,group_9026,group_9123,pheno,ST,CC
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,5,5
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,8,8
2,115,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,5,5
3,120335,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,5,5
4,120337,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,5,5


In [4]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [5]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [6]:
df['pheno']
df['pheno'].value_counts()

0    244
1     11
Name: pheno, dtype: int64

In [7]:
df.shape

(255, 1761)

In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(255, 1760)

In [60]:
df_clean.head()

,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATTTTCAAATCCCAT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,...,group_8665,group_8666,group_8667,group_8836,group_8913,group_9026,group_9123,pheno,ST,CC
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,5,5
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,8,8
2,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,5,5
3,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,5,5
4,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,5,5


In [10]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 1759) (255,)


In [11]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 224), (1, 233)]


In [12]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 244), (1, 244)]


In [13]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [23]:
pip install keras

    100% |████████████████████████████████| 378kB 5.8MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
############# Fully-Connected Neural Network ################

In [64]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [166]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [167]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [168]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 319 samples, validate on 138 samples
Epoch 1/1000
319/319 [==============================] - 0s 951us/step - loss: 5.2960 - accuracy: 0.6740 - val_loss: 0.5630 - val_accuracy: 0.7101
Epoch 2/1000
319/319 [==============================] - 0s 141us/step - loss: 0.5303 - accuracy: 0.7774 - val_loss: 0.5451 - val_accuracy: 0.7174
Epoch 3/1000
319/319 [==============================] - 0s 143us/step - loss: 0.5163 - accuracy: 0.7868 - val_loss: 0.5367 - val_accuracy: 0.7609
Epoch 4/1000
319/319 [==============================] - 0s 163us/step - loss: 0.5075 - accuracy: 0.7994 - val_loss: 0.5284 - val_accuracy: 0.7681
Epoch 5/1000
319/319 [==============================] - 0s 143us/step - loss: 0.4980 - accuracy: 0.8025 - val_loss: 0.5203 - val_accuracy: 0.7681
Epoch 6/1000
319/319 [==============================] - 0s 208us/step - loss: 0.4898 - accuracy: 0.8025 - val_loss: 0.5124 - val_accuracy: 0.7681
Epoch 7/1000
319/319 [==============================] - 0s 175us/step - loss: 

319/319 [==============================] - 0s 175us/step - loss: 0.3354 - accuracy: 0.8871 - val_loss: 0.3570 - val_accuracy: 0.8551
Epoch 57/1000
319/319 [==============================] - 0s 314us/step - loss: 0.3321 - accuracy: 0.8871 - val_loss: 0.3540 - val_accuracy: 0.8551
Epoch 58/1000
319/319 [==============================] - 0s 285us/step - loss: 0.3276 - accuracy: 0.8840 - val_loss: 0.3461 - val_accuracy: 0.8551
Epoch 59/1000
319/319 [==============================] - 0s 175us/step - loss: 0.3201 - accuracy: 0.8840 - val_loss: 0.3436 - val_accuracy: 0.8551
Epoch 60/1000
319/319 [==============================] - 0s 199us/step - loss: 0.3165 - accuracy: 0.8871 - val_loss: 0.3400 - val_accuracy: 0.8551
Epoch 61/1000
319/319 [==============================] - 0s 202us/step - loss: 0.3165 - accuracy: 0.8871 - val_loss: 0.3338 - val_accuracy: 0.8551
Epoch 62/1000
319/319 [==============================] - 0s 148us/step - loss: 0.6081 - accuracy: 0.8903 - val_loss: 0.4636 - val_ac

Epoch 112/1000
319/319 [==============================] - 0s 123us/step - loss: 0.2668 - accuracy: 0.8903 - val_loss: 0.2872 - val_accuracy: 0.8406
Epoch 113/1000
319/319 [==============================] - 0s 106us/step - loss: 0.2816 - accuracy: 0.8903 - val_loss: 0.2895 - val_accuracy: 0.8478
Epoch 114/1000
319/319 [==============================] - 0s 268us/step - loss: 0.2606 - accuracy: 0.8809 - val_loss: 0.2928 - val_accuracy: 0.8623
Epoch 115/1000
319/319 [==============================] - 0s 113us/step - loss: 0.2661 - accuracy: 0.8871 - val_loss: 0.2791 - val_accuracy: 0.8623
Epoch 116/1000
319/319 [==============================] - 0s 159us/step - loss: 0.2567 - accuracy: 0.8903 - val_loss: 0.2838 - val_accuracy: 0.8478
Epoch 117/1000
319/319 [==============================] - 0s 165us/step - loss: 0.2627 - accuracy: 0.8903 - val_loss: 0.2871 - val_accuracy: 0.8478
Epoch 118/1000
319/319 [==============================] - 0s 112us/step - loss: 0.2542 - accuracy: 0.8840 - val_

319/319 [==============================] - 0s 190us/step - loss: 0.2151 - accuracy: 0.8966 - val_loss: 0.1994 - val_accuracy: 0.9638
Epoch 168/1000
319/319 [==============================] - 0s 169us/step - loss: 0.2297 - accuracy: 0.9122 - val_loss: 0.1993 - val_accuracy: 0.9710
Epoch 169/1000
319/319 [==============================] - 0s 116us/step - loss: 0.2191 - accuracy: 0.9122 - val_loss: 0.2571 - val_accuracy: 0.8478
Epoch 170/1000
319/319 [==============================] - 0s 119us/step - loss: 0.2293 - accuracy: 0.8934 - val_loss: 0.2061 - val_accuracy: 0.9130
Epoch 171/1000
319/319 [==============================] - 0s 136us/step - loss: 0.2135 - accuracy: 0.9060 - val_loss: 0.2342 - val_accuracy: 0.8768
Epoch 172/1000
319/319 [==============================] - 0s 126us/step - loss: 0.2547 - accuracy: 0.9091 - val_loss: 0.2838 - val_accuracy: 0.8478
Epoch 173/1000
319/319 [==============================] - 0s 201us/step - loss: 0.2464 - accuracy: 0.8840 - val_loss: 0.2717 - 

319/319 [==============================] - 0s 144us/step - loss: 0.2385 - accuracy: 0.8871 - val_loss: 0.2723 - val_accuracy: 0.8551
Epoch 223/1000
319/319 [==============================] - 0s 126us/step - loss: 0.2379 - accuracy: 0.8871 - val_loss: 0.2750 - val_accuracy: 0.8551
Epoch 224/1000
319/319 [==============================] - 0s 142us/step - loss: 0.2371 - accuracy: 0.8871 - val_loss: 0.2627 - val_accuracy: 0.8551
Epoch 225/1000
319/319 [==============================] - 0s 174us/step - loss: 0.2350 - accuracy: 0.8871 - val_loss: 0.2574 - val_accuracy: 0.8551
Epoch 226/1000
319/319 [==============================] - 0s 160us/step - loss: 0.2330 - accuracy: 0.8871 - val_loss: 0.2503 - val_accuracy: 0.8768
Epoch 227/1000
319/319 [==============================] - 0s 159us/step - loss: 0.2376 - accuracy: 0.8809 - val_loss: 0.2741 - val_accuracy: 0.8551
Epoch 228/1000
319/319 [==============================] - 0s 123us/step - loss: 0.2384 - accuracy: 0.8840 - val_loss: 0.2724 - 

...

319/319 [==============================] - 0s 119us/step - loss: 0.2260 - accuracy: 0.8871 - val_loss: 0.2516 - val_accuracy: 0.8551
Epoch 773/1000
319/319 [==============================] - 0s 114us/step - loss: 0.2258 - accuracy: 0.8871 - val_loss: 0.2510 - val_accuracy: 0.8551
Epoch 774/1000
319/319 [==============================] - 0s 116us/step - loss: 0.2256 - accuracy: 0.8871 - val_loss: 0.2504 - val_accuracy: 0.8551
Epoch 775/1000
319/319 [==============================] - 0s 122us/step - loss: 0.2254 - accuracy: 0.8871 - val_loss: 0.2499 - val_accuracy: 0.8551
Epoch 776/1000
319/319 [==============================] - 0s 104us/step - loss: 0.2252 - accuracy: 0.8871 - val_loss: 0.2495 - val_accuracy: 0.8551
Epoch 777/1000
319/319 [==============================] - 0s 109us/step - loss: 0.2250 - accuracy: 0.8871 - val_loss: 0.2503 - val_accuracy: 0.8551
Epoch 778/1000
319/319 [==============================] - 0s 177us/step - loss: 0.2260 - accuracy: 0.8871 - val_loss: 0.2478 - 

319/319 [==============================] - 0s 129us/step - loss: 0.2261 - accuracy: 0.8871 - val_loss: 0.2526 - val_accuracy: 0.8551
Epoch 828/1000
319/319 [==============================] - 0s 118us/step - loss: 0.2260 - accuracy: 0.8871 - val_loss: 0.2524 - val_accuracy: 0.8551
Epoch 829/1000
319/319 [==============================] - 0s 112us/step - loss: 0.2260 - accuracy: 0.8871 - val_loss: 0.2524 - val_accuracy: 0.8551
Epoch 830/1000
319/319 [==============================] - 0s 104us/step - loss: 0.2260 - accuracy: 0.8871 - val_loss: 0.2523 - val_accuracy: 0.8551
Epoch 831/1000
319/319 [==============================] - 0s 115us/step - loss: 0.2260 - accuracy: 0.8871 - val_loss: 0.2523 - val_accuracy: 0.8551
Epoch 832/1000
319/319 [==============================] - 0s 110us/step - loss: 0.2259 - accuracy: 0.8871 - val_loss: 0.2523 - val_accuracy: 0.8551
Epoch 833/1000
319/319 [==============================] - 0s 139us/step - loss: 0.2259 - accuracy: 0.8871 - val_loss: 0.2522 - 

319/319 [==============================] - 0s 110us/step - loss: 0.2259 - accuracy: 0.8871 - val_loss: 0.2519 - val_accuracy: 0.8551
Epoch 883/1000
319/319 [==============================] - 0s 109us/step - loss: 0.2259 - accuracy: 0.8871 - val_loss: 0.2518 - val_accuracy: 0.8551
Epoch 884/1000
319/319 [==============================] - 0s 112us/step - loss: 0.2259 - accuracy: 0.8871 - val_loss: 0.2518 - val_accuracy: 0.8551
Epoch 885/1000
319/319 [==============================] - 0s 108us/step - loss: 0.2258 - accuracy: 0.8871 - val_loss: 0.2518 - val_accuracy: 0.8551
Epoch 886/1000
319/319 [==============================] - 0s 158us/step - loss: 0.2258 - accuracy: 0.8871 - val_loss: 0.2518 - val_accuracy: 0.8551
Epoch 887/1000
319/319 [==============================] - 0s 102us/step - loss: 0.2258 - accuracy: 0.8871 - val_loss: 0.2517 - val_accuracy: 0.8551
Epoch 888/1000
319/319 [==============================] - 0s 103us/step - loss: 0.2258 - accuracy: 0.8871 - val_loss: 0.2517 - 

319/319 [==============================] - 0s 110us/step - loss: 0.2258 - accuracy: 0.8871 - val_loss: 0.2515 - val_accuracy: 0.8551
Epoch 938/1000
319/319 [==============================] - 0s 110us/step - loss: 0.2258 - accuracy: 0.8871 - val_loss: 0.2515 - val_accuracy: 0.8551
Epoch 939/1000
319/319 [==============================] - 0s 106us/step - loss: 0.2258 - accuracy: 0.8871 - val_loss: 0.2515 - val_accuracy: 0.8551
Epoch 940/1000
319/319 [==============================] - 0s 113us/step - loss: 0.2258 - accuracy: 0.8871 - val_loss: 0.2515 - val_accuracy: 0.8551
Epoch 941/1000
319/319 [==============================] - 0s 121us/step - loss: 0.2258 - accuracy: 0.8871 - val_loss: 0.2515 - val_accuracy: 0.8551
Epoch 942/1000
319/319 [==============================] - 0s 108us/step - loss: 0.2258 - accuracy: 0.8871 - val_loss: 0.2515 - val_accuracy: 0.8551
Epoch 943/1000
319/319 [==============================] - 0s 103us/step - loss: 0.2258 - accuracy: 0.8871 - val_loss: 0.2515 - 

319/319 [==============================] - 0s 107us/step - loss: 0.2257 - accuracy: 0.8871 - val_loss: 0.2513 - val_accuracy: 0.8551
Epoch 993/1000
319/319 [==============================] - 0s 102us/step - loss: 0.2257 - accuracy: 0.8871 - val_loss: 0.2513 - val_accuracy: 0.8551
Epoch 994/1000
319/319 [==============================] - 0s 105us/step - loss: 0.2257 - accuracy: 0.8871 - val_loss: 0.2513 - val_accuracy: 0.8551
Epoch 995/1000
319/319 [==============================] - 0s 102us/step - loss: 0.2257 - accuracy: 0.8871 - val_loss: 0.2513 - val_accuracy: 0.8551
Epoch 996/1000
319/319 [==============================] - 0s 103us/step - loss: 0.2257 - accuracy: 0.8871 - val_loss: 0.2512 - val_accuracy: 0.8551
Epoch 997/1000
319/319 [==============================] - 0s 112us/step - loss: 0.2257 - accuracy: 0.8871 - val_loss: 0.2512 - val_accuracy: 0.8551
Epoch 998/1000
319/319 [==============================] - 0s 143us/step - loss: 0.2257 - accuracy: 0.8871 - val_loss: 0.2512 - 

In [169]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

138/138 [==============================] - 0s 107us/step
combination test accuracy: 85.51%


In [162]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [163]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [164]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 341 samples, validate on 147 samples
Epoch 1/1000
341/341 [==============================] - 0s 437us/step - loss: 4.0888 - accuracy: 0.6862 - val_loss: 0.6101 - val_accuracy: 0.7211
Epoch 2/1000
341/341 [==============================] - 0s 125us/step - loss: 0.6235 - accuracy: 0.7214 - val_loss: 0.6029 - val_accuracy: 0.6871
Epoch 3/1000
341/341 [==============================] - 0s 133us/step - loss: 0.5940 - accuracy: 0.7302 - val_loss: 0.5973 - val_accuracy: 0.6667
Epoch 4/1000
341/341 [==============================] - 0s 125us/step - loss: 0.5822 - accuracy: 0.7390 - val_loss: 0.5734 - val_accuracy: 0.7279
Epoch 5/1000
341/341 [==============================] - 0s 163us/step - loss: 0.5677 - accuracy: 0.7625 - val_loss: 0.5630 - val_accuracy: 0.7347
Epoch 6/1000
341/341 [==============================] - 0s 129us/step - loss: 0.5609 - accuracy: 0.7566 - val_loss: 0.5516 - val_accuracy: 0.7551
Epoch 7/1000
341/341 [==============================] - 0s 199us/step - loss: 

341/341 [==============================] - 0s 164us/step - loss: 0.3669 - accuracy: 0.8710 - val_loss: 0.3758 - val_accuracy: 0.8571
Epoch 57/1000
341/341 [==============================] - 0s 123us/step - loss: 0.3681 - accuracy: 0.8710 - val_loss: 0.3912 - val_accuracy: 0.8639
Epoch 58/1000
341/341 [==============================] - 0s 116us/step - loss: 0.3685 - accuracy: 0.8592 - val_loss: 0.3708 - val_accuracy: 0.8571
Epoch 59/1000
341/341 [==============================] - 0s 129us/step - loss: 0.3638 - accuracy: 0.8710 - val_loss: 0.3575 - val_accuracy: 0.8571
Epoch 60/1000
341/341 [==============================] - 0s 101us/step - loss: 0.3597 - accuracy: 0.8710 - val_loss: 0.3597 - val_accuracy: 0.8571
Epoch 61/1000
341/341 [==============================] - 0s 132us/step - loss: 0.3597 - accuracy: 0.8710 - val_loss: 0.3533 - val_accuracy: 0.8571
Epoch 62/1000
341/341 [==============================] - 0s 122us/step - loss: 0.3597 - accuracy: 0.8710 - val_loss: 0.3540 - val_ac

Epoch 112/1000
341/341 [==============================] - 0s 106us/step - loss: 0.3162 - accuracy: 0.8739 - val_loss: 0.3185 - val_accuracy: 0.8571
Epoch 113/1000
341/341 [==============================] - 0s 101us/step - loss: 0.3170 - accuracy: 0.8739 - val_loss: 0.3180 - val_accuracy: 0.8571
Epoch 114/1000
341/341 [==============================] - 0s 101us/step - loss: 0.3187 - accuracy: 0.8710 - val_loss: 0.3324 - val_accuracy: 0.8639
Epoch 115/1000
341/341 [==============================] - 0s 156us/step - loss: 0.3162 - accuracy: 0.8739 - val_loss: 0.3171 - val_accuracy: 0.8571
Epoch 116/1000
341/341 [==============================] - 0s 110us/step - loss: 0.3189 - accuracy: 0.8680 - val_loss: 0.3168 - val_accuracy: 0.8571
Epoch 117/1000
341/341 [==============================] - 0s 140us/step - loss: 0.3167 - accuracy: 0.8710 - val_loss: 0.3172 - val_accuracy: 0.8571
Epoch 118/1000
341/341 [==============================] - 0s 108us/step - loss: 0.3226 - accuracy: 0.8622 - val_

341/341 [==============================] - 0s 205us/step - loss: 0.3038 - accuracy: 0.8710 - val_loss: 0.3074 - val_accuracy: 0.8571
Epoch 168/1000
341/341 [==============================] - 0s 105us/step - loss: 0.3011 - accuracy: 0.8739 - val_loss: 0.3068 - val_accuracy: 0.8571
Epoch 169/1000
341/341 [==============================] - 0s 98us/step - loss: 0.3058 - accuracy: 0.8739 - val_loss: 0.3071 - val_accuracy: 0.8571
Epoch 170/1000
341/341 [==============================] - 0s 104us/step - loss: 0.2994 - accuracy: 0.8739 - val_loss: 0.3118 - val_accuracy: 0.8571
Epoch 171/1000
341/341 [==============================] - 0s 102us/step - loss: 0.3057 - accuracy: 0.8680 - val_loss: 0.3083 - val_accuracy: 0.8571
Epoch 172/1000
341/341 [==============================] - 0s 103us/step - loss: 0.3064 - accuracy: 0.8710 - val_loss: 0.3085 - val_accuracy: 0.8571
Epoch 173/1000
341/341 [==============================] - 0s 108us/step - loss: 0.3047 - accuracy: 0.8768 - val_loss: 0.3111 - v

341/341 [==============================] - 0s 104us/step - loss: 0.3186 - accuracy: 0.8504 - val_loss: 0.3034 - val_accuracy: 0.8571
Epoch 223/1000
341/341 [==============================] - 0s 107us/step - loss: 0.2931 - accuracy: 0.8768 - val_loss: 0.3036 - val_accuracy: 0.8571
Epoch 224/1000
341/341 [==============================] - 0s 109us/step - loss: 0.2996 - accuracy: 0.8710 - val_loss: 0.3038 - val_accuracy: 0.8571
Epoch 225/1000
341/341 [==============================] - 0s 113us/step - loss: 0.2938 - accuracy: 0.8768 - val_loss: 0.3265 - val_accuracy: 0.8639
Epoch 226/1000
341/341 [==============================] - 0s 111us/step - loss: 0.3036 - accuracy: 0.8710 - val_loss: 0.3035 - val_accuracy: 0.8571
Epoch 227/1000
341/341 [==============================] - 0s 137us/step - loss: 0.2934 - accuracy: 0.8768 - val_loss: 0.3200 - val_accuracy: 0.8639
Epoch 228/1000
341/341 [==============================] - 0s 101us/step - loss: 0.2945 - accuracy: 0.8768 - val_loss: 0.3036 - 

...

341/341 [==============================] - 0s 109us/step - loss: 0.2751 - accuracy: 0.8827 - val_loss: 0.2888 - val_accuracy: 0.8639
Epoch 773/1000
341/341 [==============================] - 0s 132us/step - loss: 0.2747 - accuracy: 0.8827 - val_loss: 0.2894 - val_accuracy: 0.8639
Epoch 774/1000
341/341 [==============================] - 0s 99us/step - loss: 0.2754 - accuracy: 0.8827 - val_loss: 0.2888 - val_accuracy: 0.8639
Epoch 775/1000
341/341 [==============================] - 0s 102us/step - loss: 0.2767 - accuracy: 0.8827 - val_loss: 0.2899 - val_accuracy: 0.8639
Epoch 776/1000
341/341 [==============================] - 0s 105us/step - loss: 0.2756 - accuracy: 0.8827 - val_loss: 0.2887 - val_accuracy: 0.8639
Epoch 777/1000
341/341 [==============================] - 0s 135us/step - loss: 0.2751 - accuracy: 0.8827 - val_loss: 0.2897 - val_accuracy: 0.8639
Epoch 778/1000
341/341 [==============================] - 0s 103us/step - loss: 0.2746 - accuracy: 0.8827 - val_loss: 0.2887 - v

341/341 [==============================] - 0s 104us/step - loss: 0.2743 - accuracy: 0.8827 - val_loss: 0.2883 - val_accuracy: 0.8639
Epoch 828/1000
341/341 [==============================] - 0s 111us/step - loss: 0.2744 - accuracy: 0.8827 - val_loss: 0.2891 - val_accuracy: 0.8639
Epoch 829/1000
341/341 [==============================] - 0s 108us/step - loss: 0.2746 - accuracy: 0.8827 - val_loss: 0.2886 - val_accuracy: 0.8639
Epoch 830/1000
341/341 [==============================] - 0s 110us/step - loss: 0.2741 - accuracy: 0.8827 - val_loss: 0.2883 - val_accuracy: 0.8639
Epoch 831/1000
341/341 [==============================] - 0s 135us/step - loss: 0.2745 - accuracy: 0.8827 - val_loss: 0.2882 - val_accuracy: 0.8639
Epoch 832/1000
341/341 [==============================] - 0s 105us/step - loss: 0.2740 - accuracy: 0.8827 - val_loss: 0.2886 - val_accuracy: 0.8639
Epoch 833/1000
341/341 [==============================] - 0s 180us/step - loss: 0.2745 - accuracy: 0.8827 - val_loss: 0.2883 - 

341/341 [==============================] - 0s 107us/step - loss: 0.2750 - accuracy: 0.8827 - val_loss: 0.2880 - val_accuracy: 0.8639
Epoch 883/1000
341/341 [==============================] - 0s 297us/step - loss: 0.2741 - accuracy: 0.8827 - val_loss: 0.2879 - val_accuracy: 0.8639
Epoch 884/1000
341/341 [==============================] - 0s 187us/step - loss: 0.2738 - accuracy: 0.8827 - val_loss: 0.2879 - val_accuracy: 0.8639
Epoch 885/1000
341/341 [==============================] - 0s 108us/step - loss: 0.2739 - accuracy: 0.8827 - val_loss: 0.2882 - val_accuracy: 0.8639
Epoch 886/1000
341/341 [==============================] - 0s 105us/step - loss: 0.2737 - accuracy: 0.8827 - val_loss: 0.2879 - val_accuracy: 0.8639
Epoch 887/1000
341/341 [==============================] - 0s 121us/step - loss: 0.2741 - accuracy: 0.8827 - val_loss: 0.2931 - val_accuracy: 0.8639
Epoch 888/1000
341/341 [==============================] - 0s 127us/step - loss: 0.2741 - accuracy: 0.8827 - val_loss: 0.2879 - 

341/341 [==============================] - 0s 139us/step - loss: 0.2736 - accuracy: 0.8827 - val_loss: 0.2892 - val_accuracy: 0.8639
Epoch 938/1000
341/341 [==============================] - 0s 183us/step - loss: 0.2735 - accuracy: 0.8827 - val_loss: 0.2878 - val_accuracy: 0.8639
Epoch 939/1000
341/341 [==============================] - 0s 103us/step - loss: 0.2733 - accuracy: 0.8827 - val_loss: 0.2887 - val_accuracy: 0.8639
Epoch 940/1000
341/341 [==============================] - 0s 108us/step - loss: 0.2734 - accuracy: 0.8827 - val_loss: 0.2883 - val_accuracy: 0.8639
Epoch 941/1000
341/341 [==============================] - 0s 101us/step - loss: 0.2735 - accuracy: 0.8827 - val_loss: 0.2875 - val_accuracy: 0.8639
Epoch 942/1000
341/341 [==============================] - 0s 103us/step - loss: 0.2733 - accuracy: 0.8827 - val_loss: 0.2884 - val_accuracy: 0.8639
Epoch 943/1000
341/341 [==============================] - 0s 102us/step - loss: 0.2734 - accuracy: 0.8827 - val_loss: 0.2875 - 

341/341 [==============================] - 0s 233us/step - loss: 0.2736 - accuracy: 0.8827 - val_loss: 0.2874 - val_accuracy: 0.8639
Epoch 993/1000
341/341 [==============================] - 0s 114us/step - loss: 0.2734 - accuracy: 0.8827 - val_loss: 0.2874 - val_accuracy: 0.8639
Epoch 994/1000
341/341 [==============================] - 0s 101us/step - loss: 0.2732 - accuracy: 0.8827 - val_loss: 0.2873 - val_accuracy: 0.8639
Epoch 995/1000
341/341 [==============================] - 0s 228us/step - loss: 0.2729 - accuracy: 0.8827 - val_loss: 0.2876 - val_accuracy: 0.8639
Epoch 996/1000
341/341 [==============================] - 0s 183us/step - loss: 0.2733 - accuracy: 0.8827 - val_loss: 0.2876 - val_accuracy: 0.8639
Epoch 997/1000
341/341 [==============================] - 0s 108us/step - loss: 0.2730 - accuracy: 0.8827 - val_loss: 0.2873 - val_accuracy: 0.8639
Epoch 998/1000
341/341 [==============================] - 0s 105us/step - loss: 0.2733 - accuracy: 0.8827 - val_loss: 0.2873 - 

In [165]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

147/147 [==============================] - 0s 97us/step
over-sampling test accuracy: 86.39%


In [41]:
############ Feature selection using lasso ##########

In [65]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [47]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [73]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [106]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [107]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 1759
selected features: 32


In [108]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  57,  139,  235,  284,  285,  286,  287,  338,  572,  575,  618,
         640,  688,  700,  760,  776,  805,  866,  944, 1036, 1119, 1206,
        1254, 1273, 1289, 1460, 1481, 1547, 1609, 1638, 1757, 1758]])

In [140]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTATATACTTTA', 'TTTAATATTATAAAATG', 'TTCGTATTTAGC',
       'TTATTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGA',
       'TTATTATGTTAATG',
       'TTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAA',
       'TTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGC',
       'TTACTATATTTGGCATTATATGTAGTATTATTTTCACGAGATGGG', 'TCATACATAAAAT',
       'TCATAACACAAGAGATATAGCAAAGTTTAAAAAAGTACTATAAAATAGCAATTGAATGTCCAGTAACAAATTTGGAGGAAGCGTATATGTATCAAACAAT',
       'TATTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGAT',
       'TATTAATCATGAAC',
       'TATAAGGTCATGTCACGTTGAAACAGAGGTTCCTCGGCATTTTTGAAAATAATCATTAGCTTGCTCACTATATAAT',
       'TAGTGAGCAAG',
       'TAATTGTTTGATACATATACGCTTCCTCCAAATTTGTTACTGGACATTCAATTGCTATTTTATAGTACTTTTTTAAACTTTGCTATATCTCTTGTGTTAT',
       'TAATAATTGTTTGATACATATACGCTTCCTCCAAATTTGTTACTGGACATTCAATTGCTATTTTATAGTACTTTTTTAAACTT

In [82]:
removed_features = df_features_comb.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1747, 1748, 1749, 1750, 1751, 1752, 1753, 1754, 1755, 1756],
           dtype='int64', length=1727)

In [83]:
len(removed_features)

1727

In [144]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 32) (255,) (255, 33)


In [145]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 219), (1, 199)]


In [151]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 244), (1, 244)]


In [146]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [152]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [147]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [148]:
model2_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [149]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 292 samples, validate on 126 samples
Epoch 1/1000
292/292 [==============================] - 1s 2ms/step - loss: 22.7263 - accuracy: 0.5445 - val_loss: 0.6124 - val_accuracy: 0.8333
Epoch 2/1000
292/292 [==============================] - 0s 256us/step - loss: 0.5854 - accuracy: 0.8253 - val_loss: 0.5999 - val_accuracy: 0.8333
Epoch 3/1000
292/292 [==============================] - 0s 377us/step - loss: 0.5674 - accuracy: 0.8253 - val_loss: 0.5896 - val_accuracy: 0.8175
Epoch 4/1000
292/292 [==============================] - 0s 319us/step - loss: 0.5488 - accuracy: 0.8082 - val_loss: 0.5749 - val_accuracy: 0.8175
Epoch 5/1000
292/292 [==============================] - 0s 180us/step - loss: 0.5339 - accuracy: 0.8082 - val_loss: 0.5893 - val_accuracy: 0.8095
Epoch 6/1000
292/292 [==============================] - 0s 400us/step - loss: 0.5251 - accuracy: 0.8116 - val_loss: 0.5614 - val_accuracy: 0.8175
Epoch 7/1000
292/292 [==============================] - 0s 707us/step - loss: 0

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.110259). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


292/292 [==============================] - 0s 380us/step - loss: 0.4153 - accuracy: 0.8356 - val_loss: 0.4218 - val_accuracy: 0.8254
Epoch 31/1000
292/292 [==============================] - 0s 283us/step - loss: 0.4138 - accuracy: 0.8390 - val_loss: 0.4152 - val_accuracy: 0.8333
Epoch 32/1000
292/292 [==============================] - 0s 204us/step - loss: 0.4108 - accuracy: 0.8390 - val_loss: 0.4193 - val_accuracy: 0.8254
Epoch 33/1000
292/292 [==============================] - 0s 172us/step - loss: 0.4088 - accuracy: 0.8390 - val_loss: 0.4183 - val_accuracy: 0.8254
Epoch 34/1000
292/292 [==============================] - 0s 332us/step - loss: 0.4076 - accuracy: 0.8390 - val_loss: 0.4138 - val_accuracy: 0.8254
Epoch 35/1000
292/292 [==============================] - 0s 441us/step - loss: 0.4052 - accuracy: 0.8390 - val_loss: 0.4099 - val_accuracy: 0.8254
Epoch 36/1000
292/292 [==============================] - 0s 189us/step - loss: 0.4038 - accuracy: 0.8390 - val_loss: 0.4146 - val_ac

Epoch 86/1000
292/292 [==============================] - 0s 141us/step - loss: 0.3546 - accuracy: 0.8664 - val_loss: 0.3548 - val_accuracy: 0.8492
Epoch 87/1000
292/292 [==============================] - 0s 382us/step - loss: 0.3524 - accuracy: 0.8664 - val_loss: 0.3598 - val_accuracy: 0.8492
Epoch 88/1000
292/292 [==============================] - 0s 289us/step - loss: 0.3528 - accuracy: 0.8664 - val_loss: 0.3544 - val_accuracy: 0.8571
Epoch 89/1000
292/292 [==============================] - 0s 125us/step - loss: 0.3517 - accuracy: 0.8664 - val_loss: 0.3575 - val_accuracy: 0.8492
Epoch 90/1000
292/292 [==============================] - 0s 123us/step - loss: 0.3526 - accuracy: 0.8664 - val_loss: 0.3589 - val_accuracy: 0.8492
Epoch 91/1000
292/292 [==============================] - 0s 102us/step - loss: 0.3508 - accuracy: 0.8664 - val_loss: 0.3561 - val_accuracy: 0.8492
Epoch 92/1000
292/292 [==============================] - 0s 98us/step - loss: 0.3497 - accuracy: 0.8664 - val_loss: 0.

Epoch 142/1000
292/292 [==============================] - 0s 90us/step - loss: 0.3471 - accuracy: 0.8596 - val_loss: 0.3352 - val_accuracy: 0.8571
Epoch 143/1000
292/292 [==============================] - 0s 79us/step - loss: 0.3464 - accuracy: 0.8596 - val_loss: 0.3417 - val_accuracy: 0.8571
Epoch 144/1000
292/292 [==============================] - 0s 78us/step - loss: 0.3466 - accuracy: 0.8596 - val_loss: 0.3343 - val_accuracy: 0.8571
Epoch 145/1000
292/292 [==============================] - 0s 81us/step - loss: 0.3450 - accuracy: 0.8630 - val_loss: 0.3358 - val_accuracy: 0.8571
Epoch 146/1000
292/292 [==============================] - 0s 84us/step - loss: 0.3444 - accuracy: 0.8596 - val_loss: 0.3457 - val_accuracy: 0.8571
Epoch 147/1000
292/292 [==============================] - 0s 82us/step - loss: 0.3449 - accuracy: 0.8596 - val_loss: 0.3374 - val_accuracy: 0.8571
Epoch 148/1000
292/292 [==============================] - 0s 98us/step - loss: 0.3441 - accuracy: 0.8630 - val_loss: 0

...

292/292 [==============================] - 0s 608us/step - loss: 0.3250 - accuracy: 0.8664 - val_loss: 0.3113 - val_accuracy: 0.8730
Epoch 860/1000
292/292 [==============================] - 0s 152us/step - loss: 0.3247 - accuracy: 0.8664 - val_loss: 0.3073 - val_accuracy: 0.8810
Epoch 861/1000
292/292 [==============================] - 0s 165us/step - loss: 0.3251 - accuracy: 0.8664 - val_loss: 0.3129 - val_accuracy: 0.8730
Epoch 862/1000
292/292 [==============================] - 0s 185us/step - loss: 0.3254 - accuracy: 0.8664 - val_loss: 0.3116 - val_accuracy: 0.8730
Epoch 863/1000
292/292 [==============================] - 0s 147us/step - loss: 0.3251 - accuracy: 0.8630 - val_loss: 0.3453 - val_accuracy: 0.8571
Epoch 864/1000
292/292 [==============================] - 0s 164us/step - loss: 0.3643 - accuracy: 0.8390 - val_loss: 0.3181 - val_accuracy: 0.8730
Epoch 865/1000
292/292 [==============================] - 0s 175us/step - loss: 0.3355 - accuracy: 0.8562 - val_loss: 0.3199 - 

292/292 [==============================] - 0s 127us/step - loss: 0.3243 - accuracy: 0.8664 - val_loss: 0.3035 - val_accuracy: 0.8810
Epoch 915/1000
292/292 [==============================] - 0s 143us/step - loss: 0.3248 - accuracy: 0.8664 - val_loss: 0.3042 - val_accuracy: 0.8810
Epoch 916/1000
292/292 [==============================] - 0s 129us/step - loss: 0.3242 - accuracy: 0.8664 - val_loss: 0.3101 - val_accuracy: 0.8730
Epoch 917/1000
292/292 [==============================] - 0s 217us/step - loss: 0.3244 - accuracy: 0.8664 - val_loss: 0.3112 - val_accuracy: 0.8730
Epoch 918/1000
292/292 [==============================] - 0s 526us/step - loss: 0.3243 - accuracy: 0.8664 - val_loss: 0.3118 - val_accuracy: 0.8730
Epoch 919/1000
292/292 [==============================] - 0s 92us/step - loss: 0.3246 - accuracy: 0.8664 - val_loss: 0.3105 - val_accuracy: 0.8730
Epoch 920/1000
292/292 [==============================] - 0s 94us/step - loss: 0.3239 - accuracy: 0.8664 - val_loss: 0.3049 - va

292/292 [==============================] - 0s 123us/step - loss: 0.3242 - accuracy: 0.8664 - val_loss: 0.3051 - val_accuracy: 0.8810
Epoch 970/1000
292/292 [==============================] - 0s 119us/step - loss: 0.3245 - accuracy: 0.8664 - val_loss: 0.3060 - val_accuracy: 0.8810
Epoch 971/1000
292/292 [==============================] - 0s 74us/step - loss: 0.3243 - accuracy: 0.8664 - val_loss: 0.3049 - val_accuracy: 0.8810
Epoch 972/1000
292/292 [==============================] - 0s 86us/step - loss: 0.3245 - accuracy: 0.8664 - val_loss: 0.3084 - val_accuracy: 0.8730
Epoch 973/1000
292/292 [==============================] - 0s 163us/step - loss: 0.3245 - accuracy: 0.8664 - val_loss: 0.3100 - val_accuracy: 0.8730
Epoch 974/1000
292/292 [==============================] - 0s 84us/step - loss: 0.3239 - accuracy: 0.8664 - val_loss: 0.3030 - val_accuracy: 0.8810
Epoch 975/1000
292/292 [==============================] - 0s 93us/step - loss: 0.3246 - accuracy: 0.8664 - val_loss: 0.4338 - val_

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.182179). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


Epoch 995/1000
292/292 [==============================] - 0s 251us/step - loss: 0.3286 - accuracy: 0.8630 - val_loss: 0.3097 - val_accuracy: 0.8810
Epoch 996/1000
292/292 [==============================] - 0s 389us/step - loss: 0.3277 - accuracy: 0.8664 - val_loss: 0.3116 - val_accuracy: 0.8730
Epoch 997/1000
292/292 [==============================] - 0s 116us/step - loss: 0.3269 - accuracy: 0.8664 - val_loss: 0.3164 - val_accuracy: 0.8571
Epoch 998/1000
292/292 [==============================] - 0s 490us/step - loss: 0.3276 - accuracy: 0.8630 - val_loss: 0.3078 - val_accuracy: 0.8810
Epoch 999/1000
292/292 [==============================] - 0s 237us/step - loss: 0.3266 - accuracy: 0.8664 - val_loss: 0.3163 - val_accuracy: 0.8571
Epoch 1000/1000
292/292 [==============================] - 0s 208us/step - loss: 0.3264 - accuracy: 0.8630 - val_loss: 0.3095 - val_accuracy: 0.8730


In [150]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))


126/126 [==============================] - 0s 88us/step
combination test accuracy: 87.30%


In [170]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [171]:
model2_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [172]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 341 samples, validate on 147 samples
Epoch 1/1000
341/341 [==============================] - 0s 1ms/step - loss: 29.0129 - accuracy: 0.4135 - val_loss: 0.6910 - val_accuracy: 0.5374
Epoch 2/1000
341/341 [==============================] - 0s 255us/step - loss: 0.6868 - accuracy: 0.5748 - val_loss: 0.6724 - val_accuracy: 0.6531
Epoch 3/1000
341/341 [==============================] - 0s 204us/step - loss: 0.6722 - accuracy: 0.6862 - val_loss: 0.6718 - val_accuracy: 0.6803
Epoch 4/1000
341/341 [==============================] - 0s 109us/step - loss: 0.6591 - accuracy: 0.7126 - val_loss: 0.6490 - val_accuracy: 0.6735
Epoch 5/1000
341/341 [==============================] - 0s 99us/step - loss: 0.6499 - accuracy: 0.7009 - val_loss: 0.6497 - val_accuracy: 0.6327
Epoch 6/1000
341/341 [==============================] - 0s 135us/step - loss: 0.6459 - accuracy: 0.6950 - val_loss: 0.6556 - val_accuracy: 0.6735
Epoch 7/1000
341/341 [==============================] - 0s 96us/step - loss: 0.6

341/341 [==============================] - 0s 124us/step - loss: 0.5375 - accuracy: 0.7478 - val_loss: 0.5379 - val_accuracy: 0.7755
Epoch 57/1000
341/341 [==============================] - 0s 93us/step - loss: 0.5359 - accuracy: 0.7595 - val_loss: 0.5373 - val_accuracy: 0.7755
Epoch 58/1000
341/341 [==============================] - 0s 84us/step - loss: 0.5346 - accuracy: 0.7654 - val_loss: 0.5389 - val_accuracy: 0.7347
Epoch 59/1000
341/341 [==============================] - 0s 83us/step - loss: 0.5341 - accuracy: 0.7566 - val_loss: 0.5399 - val_accuracy: 0.6667
Epoch 60/1000
341/341 [==============================] - 0s 85us/step - loss: 0.5340 - accuracy: 0.7742 - val_loss: 0.5366 - val_accuracy: 0.7347
Epoch 61/1000
341/341 [==============================] - 0s 87us/step - loss: 0.5356 - accuracy: 0.7507 - val_loss: 0.5363 - val_accuracy: 0.7891
Epoch 62/1000
341/341 [==============================] - 0s 114us/step - loss: 0.5315 - accuracy: 0.7625 - val_loss: 0.5327 - val_accurac

341/341 [==============================] - 0s 81us/step - loss: 0.4940 - accuracy: 0.7830 - val_loss: 0.4970 - val_accuracy: 0.7823
Epoch 113/1000
341/341 [==============================] - 0s 126us/step - loss: 0.4929 - accuracy: 0.7918 - val_loss: 0.4992 - val_accuracy: 0.7891
Epoch 114/1000
341/341 [==============================] - 0s 130us/step - loss: 0.4950 - accuracy: 0.7889 - val_loss: 0.5044 - val_accuracy: 0.7891
Epoch 115/1000
341/341 [==============================] - 0s 84us/step - loss: 0.4993 - accuracy: 0.7771 - val_loss: 0.4947 - val_accuracy: 0.7823
Epoch 116/1000
341/341 [==============================] - 0s 107us/step - loss: 0.4939 - accuracy: 0.7830 - val_loss: 0.5152 - val_accuracy: 0.7211
Epoch 117/1000
341/341 [==============================] - 0s 112us/step - loss: 0.4960 - accuracy: 0.7859 - val_loss: 0.4973 - val_accuracy: 0.7823
Epoch 118/1000
341/341 [==============================] - 0s 102us/step - loss: 0.4885 - accuracy: 0.7859 - val_loss: 0.5112 - va

Epoch 168/1000
341/341 [==============================] - 0s 271us/step - loss: 0.4687 - accuracy: 0.7918 - val_loss: 0.5020 - val_accuracy: 0.7279
Epoch 169/1000
341/341 [==============================] - 0s 124us/step - loss: 0.4688 - accuracy: 0.7742 - val_loss: 0.4666 - val_accuracy: 0.7891
Epoch 170/1000
341/341 [==============================] - 0s 94us/step - loss: 0.4661 - accuracy: 0.7947 - val_loss: 0.4679 - val_accuracy: 0.7891
Epoch 171/1000
341/341 [==============================] - 0s 108us/step - loss: 0.4732 - accuracy: 0.7889 - val_loss: 0.4694 - val_accuracy: 0.7891
Epoch 172/1000
341/341 [==============================] - 0s 90us/step - loss: 0.4629 - accuracy: 0.7918 - val_loss: 0.4715 - val_accuracy: 0.7891
Epoch 173/1000
341/341 [==============================] - 0s 93us/step - loss: 0.4700 - accuracy: 0.7918 - val_loss: 0.4650 - val_accuracy: 0.7891
Epoch 174/1000
341/341 [==============================] - 0s 230us/step - loss: 0.4751 - accuracy: 0.7830 - val_los

341/341 [==============================] - 0s 99us/step - loss: 0.4448 - accuracy: 0.7947 - val_loss: 0.4444 - val_accuracy: 0.7891
Epoch 224/1000
341/341 [==============================] - 0s 99us/step - loss: 0.4462 - accuracy: 0.7918 - val_loss: 0.4435 - val_accuracy: 0.7891
Epoch 225/1000
341/341 [==============================] - 0s 81us/step - loss: 0.4447 - accuracy: 0.7947 - val_loss: 0.4567 - val_accuracy: 0.7959
Epoch 226/1000
341/341 [==============================] - 0s 90us/step - loss: 0.4440 - accuracy: 0.7947 - val_loss: 0.4430 - val_accuracy: 0.7891
Epoch 227/1000
341/341 [==============================] - 0s 84us/step - loss: 0.4424 - accuracy: 0.7947 - val_loss: 0.4401 - val_accuracy: 0.7891
Epoch 228/1000
341/341 [==============================] - 0s 129us/step - loss: 0.4481 - accuracy: 0.7801 - val_loss: 0.4862 - val_accuracy: 0.7279
Epoch 229/1000
341/341 [==============================] - 0s 95us/step - loss: 0.4455 - accuracy: 0.7947 - val_loss: 0.4912 - val_ac

...

Epoch 777/1000
341/341 [==============================] - 0s 325us/step - loss: 0.3760 - accuracy: 0.8416 - val_loss: 0.3828 - val_accuracy: 0.8367
Epoch 778/1000
341/341 [==============================] - 0s 90us/step - loss: 0.3769 - accuracy: 0.8416 - val_loss: 0.3837 - val_accuracy: 0.8367
Epoch 779/1000
341/341 [==============================] - 0s 99us/step - loss: 0.3758 - accuracy: 0.8387 - val_loss: 0.3791 - val_accuracy: 0.8367
Epoch 780/1000
341/341 [==============================] - 0s 92us/step - loss: 0.3745 - accuracy: 0.8358 - val_loss: 0.3844 - val_accuracy: 0.8299
Epoch 781/1000
341/341 [==============================] - 0s 145us/step - loss: 0.3746 - accuracy: 0.8387 - val_loss: 0.3884 - val_accuracy: 0.8367
Epoch 782/1000
341/341 [==============================] - 0s 148us/step - loss: 0.3834 - accuracy: 0.8328 - val_loss: 0.3855 - val_accuracy: 0.8299
Epoch 783/1000
341/341 [==============================] - 0s 82us/step - loss: 0.3722 - accuracy: 0.8416 - val_loss

341/341 [==============================] - 0s 65us/step - loss: 0.3715 - accuracy: 0.8446 - val_loss: 0.3745 - val_accuracy: 0.8435
Epoch 833/1000
341/341 [==============================] - 0s 101us/step - loss: 0.3656 - accuracy: 0.8534 - val_loss: 0.3719 - val_accuracy: 0.8435
Epoch 834/1000
341/341 [==============================] - 0s 76us/step - loss: 0.3649 - accuracy: 0.8446 - val_loss: 0.3765 - val_accuracy: 0.8367
Epoch 835/1000
341/341 [==============================] - 0s 80us/step - loss: 0.3653 - accuracy: 0.8504 - val_loss: 0.3690 - val_accuracy: 0.8435
Epoch 836/1000
341/341 [==============================] - 0s 133us/step - loss: 0.3697 - accuracy: 0.8416 - val_loss: 0.3704 - val_accuracy: 0.8435
Epoch 837/1000
341/341 [==============================] - 0s 96us/step - loss: 0.3628 - accuracy: 0.8475 - val_loss: 0.3687 - val_accuracy: 0.8435
Epoch 838/1000
341/341 [==============================] - 0s 147us/step - loss: 0.3686 - accuracy: 0.8416 - val_loss: 0.3785 - val_

341/341 [==============================] - 0s 115us/step - loss: 0.3741 - accuracy: 0.8416 - val_loss: 0.3891 - val_accuracy: 0.8367
Epoch 888/1000
341/341 [==============================] - 0s 177us/step - loss: 0.3854 - accuracy: 0.8446 - val_loss: 0.3912 - val_accuracy: 0.8299
Epoch 889/1000
341/341 [==============================] - 0s 115us/step - loss: 0.3836 - accuracy: 0.8270 - val_loss: 0.3826 - val_accuracy: 0.8367
Epoch 890/1000
341/341 [==============================] - 0s 179us/step - loss: 0.3755 - accuracy: 0.8416 - val_loss: 0.3805 - val_accuracy: 0.8367
Epoch 891/1000
341/341 [==============================] - 0s 194us/step - loss: 0.3743 - accuracy: 0.8387 - val_loss: 0.3873 - val_accuracy: 0.8299
Epoch 892/1000
341/341 [==============================] - 0s 133us/step - loss: 0.3721 - accuracy: 0.8446 - val_loss: 0.3787 - val_accuracy: 0.8367
Epoch 893/1000
341/341 [==============================] - 0s 103us/step - loss: 0.3748 - accuracy: 0.8358 - val_loss: 0.3789 - 

341/341 [==============================] - 0s 88us/step - loss: 0.3736 - accuracy: 0.8358 - val_loss: 0.4391 - val_accuracy: 0.7755
Epoch 943/1000
341/341 [==============================] - 0s 99us/step - loss: 0.3722 - accuracy: 0.8416 - val_loss: 0.4010 - val_accuracy: 0.8095
Epoch 944/1000
341/341 [==============================] - 0s 97us/step - loss: 0.3815 - accuracy: 0.8270 - val_loss: 0.4129 - val_accuracy: 0.8095
Epoch 945/1000
341/341 [==============================] - 0s 172us/step - loss: 0.3803 - accuracy: 0.8358 - val_loss: 0.3978 - val_accuracy: 0.8163
Epoch 946/1000
341/341 [==============================] - 0s 148us/step - loss: 0.3662 - accuracy: 0.8446 - val_loss: 0.4575 - val_accuracy: 0.7347
Epoch 947/1000
341/341 [==============================] - 0s 151us/step - loss: 0.3720 - accuracy: 0.8416 - val_loss: 0.4124 - val_accuracy: 0.8095
Epoch 948/1000
341/341 [==============================] - 0s 99us/step - loss: 0.3668 - accuracy: 0.8446 - val_loss: 0.4352 - val_

Epoch 998/1000
341/341 [==============================] - 0s 148us/step - loss: 0.3856 - accuracy: 0.8416 - val_loss: 0.3928 - val_accuracy: 0.8299
Epoch 999/1000
341/341 [==============================] - 0s 72us/step - loss: 0.3867 - accuracy: 0.8416 - val_loss: 0.3994 - val_accuracy: 0.8231
Epoch 1000/1000
341/341 [==============================] - 0s 85us/step - loss: 0.3832 - accuracy: 0.8416 - val_loss: 0.4045 - val_accuracy: 0.8231


In [173]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over2*100))

147/147 [==============================] - 0s 112us/step
combination test accuracy: 82.31%


In [ ]:
############ Logistic Regression ############

In [174]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [175]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [176]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 100.00%


In [177]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [178]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 100.00%


In [100]:
############## Random Forest ##############

[1.        1.        0.9787234 1.        1.       ]
0.9957446808510639
0.008510638297872353


In [179]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [180]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [181]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [182]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 100.00%


In [183]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [184]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         1.         1.         1.         0.98412698]
0.9968253968253968


In [185]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.98550725 1.         0.98529412 0.98529412 0.98529412]
0.9882779198635976
